# HOP-6: if same day transaction count = transaction count, same day patient count = patient count

Description: if same day transaction count is equal to transaction count, then same day patient count should be equal to patient count.

Starting Author: Amy Jin (amy@careset.com)

Date: July 20th, 2018

https://docs.google.com/spreadsheets/d/1sltXNV5CDfkClTs23rCgOSImqeZDIjJJoTZQwtDvZ18/edit#gid=0

## Connection to Parenthood Server

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [5]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                       database name in server
# 2）table_name:                    table name
# 3) same_day_transaction_count:    same day transaction count column
# 4) transaction_count:             transaction count column 
# 5) same_day_patient_count:        same day patient count column 
# 6) patient_count:                 patient count column 
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:            PASS/FAIL
# 2) If FAIL, the test will print out all the same_day_transaction_count, transaction_count, 
# same_day_patient_count, patient_count fail the test.


def hop_6(db_name, table_name, same_day_transaction_count, \
             transaction_count, same_day_patient_count, patient_count):

    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()

        query = ('''
            SELECT *
            FROM {db}.{t1}
            WHERE ({col1} = {col2} AND {col3} <> {col4})
            LIMIT 1;
        '''.format(db = db_name, t1 = table_name, col1 = same_day_transaction_count, \
             col2 = transaction_count, col3 = same_day_patient_count, col4 = patient_count))

        cur.execute(query)
        rows = list(sum(cur.fetchall(), ()))


        if not len(rows):
            print ("Count-9 test result for {}.{} : PASS".format(db_name, table_name))
        else:
                   
            print ("Count-9 test result for {}.{} : FAIL".format(db_name, table_name) + '\n')
            print ("The following rows fail the test:" +'\n')
            print ('{}, {}, {}, {}'.format(same_day_transaction_count, transaction_count, same_day_patient_count, patient_count) +'\n')

            # MySQL query to get the failed rows
            query = ('''
                SELECT {col1}, {col2}, {col3}, {col4}
                FROM {db}.{t1}
                WHERE ({col1} = {col2} AND {col3} <> {col4});
            '''.format(db = db_name, t1 = table_name, col1 = same_day_transaction_count, \
             col2 = transaction_count, col3 = same_day_patient_count, col4 = patient_count))

            cur.execute(query)

            for row in cur.fetchall():
                for i in range(0,len(row)):
                    print (str(row[i]), end=", ")
                print ('\n')

        cur.close()
        connection.close()

## Test Example

In [6]:
hop_6('_amy', 'test_hop_directed_good', 'same_day_transaction_count', \
             'transaction_count', 'same_day_patient_count', 'patient_count')

Count-9 test result for _amy.test_hop_directed_good : PASS


In [7]:
hop_6('_amy', 'test_hop_directed_bad', 'same_day_transaction_count', \
             'transaction_count', 'same_day_patient_count', 'patient_count')

Count-9 test result for _amy.test_hop_directed_bad : FAIL

The following rows fail the test:

same_day_transaction_count, transaction_count, same_day_patient_count, patient_count

218, 218, 500, 191, 

32, 32, 500, 29, 

46, 46, 500, 42, 

115, 115, 500, 106, 

21, 21, 500, 21, 

12, 12, 500, 11, 

13, 13, 500, 13, 

180, 180, 500, 169, 

12, 12, 500, 12, 

21, 21, 500, 21, 

39, 39, 500, 39, 

46, 46, 500, 46, 

